# Superfacility API Demo


## Let's get started! This is an interactive demo of a non-interactive workflow. 
### It highlights the asynchronous nature of the API and polls the /tasks endpoint.

### Load wrappers, libraries. 
Please adjust the "training" path to wherever you copied the training materials.

In [ ]:
API_URL="https://api.nersc.gov/api/v1.2"
jwt = None

import urllib
import base64
import urllib.request
import urllib.parse
import json
import time
import os

## Adjust here ####
home = os.environ['HOME']
training = home + "/sfapi_training"
###################

class Task:
    """
    A crude implementation to work with Task
    """
    def __init__(self, task_id=0, status='',error=None, **kwargs):
        self.id = task_id
        self.status = status
        self.error= error
        self.result= None
        self._nchecked=0
        # This is ugly bc the task interface isn't uniform
        self.kwargs = kwargs
    
    def check(self):
        _ = api("tasks/"+self.id)
        #print(_, self.id)
        self.status = _['status']
        self.result = _['result']
        self._nchecked+=1
        if self.result is not None and self.result != 'ok':
            res = json.loads(self.result[self.result.find('{'):])
            #res = _['result']
            #print(res)
            if 'status' in res and res['status']=='error': # this is for jobs mostly, so not really in all Tasks
                self.status = 'failed'
                raise RuntimeError(res['error'].split('\n')[-2])
        return _
        
    def get_result(self):
        if self.status not in ['completed','failed']:
            print("Task %s is in status '%s' not yet completed" % (self.id, self.status))
            return
        elif self.status == 'completed':
            try:
                return json.loads(self.result)
            except:
                return self.result
            
    def wait_for_completed(self, dt=1.0, time_out=2):
        t0=time.time()
        while self.status not in ['completed','failed']:
            time.sleep(dt)
            _ = self.check()
            print('Poll %02d:' % self._nchecked + str(_), end='\r')
        print()
        print(self.result)
        return self.result
    
    def wait_for_result(self):
        self.wait_for_completed()
        return self.get_result()
            
   
def api(path, data=None, as_form=False):
    """Make a superfacility api call. 
    Path is a relative path w/o the leading /. 
    Data (optional) is a python dictionary."""
    global jwt
    global client
    url = "%s/%s" % (API_URL, path)
    if client.token['expires_at'] < time.time()-100:
        print("Fetching new token")
        client.fetch_token(token_url, grant_type="client_credentials")
    
    if data:
        ret = client.post(url, data).json()
    else:
        ret = client.get(url).json()
    
    if 'task_id' in ret:
        return Task(**ret)
    else:
        return ret
   

### Obtain a _JSON Web Token_ (JWT) to authenticate to the API. Make sure to fill in the json formatted credentials (.jwk) of a client with RED sec level

In [ ]:
from authlib.integrations.requests_client import OAuth2Session
from authlib.oauth2.rfc7523 import PrivateKeyJWT
from authlib.jose import JsonWebKey
import json

token_url = "https://oidc.nersc.gov/c2id/token"
# perlmutter
client_id = 'b32qotj4i6upw'
key_file_jwk = training+"/priv_key.jwk"
key_file_pem = training+"/priv_key.pem"
key_file = key_file_pem

with open(key_file_pem) as kf:
    client_secret = kf.read()
    kf.close()

client = OAuth2Session(client_id=client_id,
                        client_secret=client_secret, 
                        token_endpoint_auth_method="private_key_jwt")
client.register_client_auth_method(PrivateKeyJWT(token_url))
resp = client.fetch_token(token_url, grant_type="client_credentials")
token = resp["access_token"]
jwt = token
print(json.dumps(resp,indent=2))

### Before we start any computing, let's check which systems are up.

In [ ]:
r = client.get("https://api.nersc.gov/api/v1.2/status/perlmutter")
print(json.dumps(r.json(),indent=2))

In [ ]:
health = client.get(API_URL+"/status/perlmutter").json()
print(health['full_name'], health['status'])
health = client.get(API_URL+"/status/dtns").json()
print(health['full_name'], health['status'])
health = client.get(API_URL+"/status/community_filesystem").json()
print(health['full_name'], health['status'])
health = client.get(API_URL+"/status/globus").json()
print(health['full_name'], health['status'])

### We can also take a look into the future to better plan our work around planned outages. 

In [ ]:
planned_outages = client.get(API_URL+"/status/outages/planned/perlmutter").json()
print(json.dumps(planned_outages,indent=2))

### Now let's create a job script in the sfapi_training folder

In [ ]:
job_script = """#!/bin/bash -l

#SBATCH -q debug
#SBATCH -A nstaff
#SBATCH -N 1
#SBATCH -C cpu
#SBATCH --tasks-per-node=32
#SBATCH -t 00:10:00
#SBATCH --exclusive

module load python

hostname
sleep 20
srun -n 1 python -c "print('Finished')"
""" 
job_file = training + "/job.sh"
print(job_file)

In [ ]:
response = api("utilities/command/perlmutter", { "executable": 'cat > {0} << EOF\n{1}'.format(job_file,job_script)})
if isinstance(response, Task):
    print(response.wait_for_result()['output'].strip())

### Check whether the file is there

In [ ]:
ls_results = api("utilities/ls/perlmutter/" + job_file)
print(json.dumps(ls_results, indent=2))

### Submit to queue

In [ ]:
response = api("compute/jobs/perlmutter", { "job": job_file, "isPath":"true"})
if isinstance(response, Task):
    jobid=response.wait_for_result()['jobid'].strip()

### Check queue status, wait for job to complete.

In [ ]:
#print(json.dumps(api("compute/jobs/perlmutter/"+jobid+"?sacct=true"), indent=2))
print(api("compute/jobs/perlmutter/"+jobid+"?sacct=true")['output'][0]['state'])

### Read from the slurm output file

In [ ]:
slurmfile = home+"/slurm-"+jobid+".out"
response = api("utilities/command/perlmutter", { "executable": "tail -n 20 "+slurmfile })
if isinstance(response, Task):
    print(response.wait_for_result()['output'].strip())

## Now to YOU...Try to run your own job! Or wait for the sfapi_client demos

